# Testing NBodySimulation
This notebook initializes and runs the `NBodySimulation` model, solving for positions, velocities, and conserving energy.

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from simulation import NBodySimulation

%matplotlib inline

In [10]:
# Initialize a 3-body system
sim = NBodySimulation(
    num_particles=3,
    gravitational_constant=1.0,
    softening_length=0.1,
    integrator='rk4'
)

np.random.seed(42)
sim.initialize_random_state(
    position_scale=2.0,
    velocity_scale=0.5,
    mass_range=(0.8, 1.2)
)

init_energy = sim.compute_total_energy()
print(f"Initial Total Energy: {init_energy:.5f}")

Initial Total Energy: -0.73186


In [11]:
# Simulate the orbit
dt = 0.01
total_time = 20.0

history_positions, history_velocities, history_times = sim.simulate(
    total_time=total_time,
    dt=dt,
    save_every=5
)

final_energy = sim.compute_total_energy()
print(f"Final Total Energy: {final_energy:.5f}")
print(f"Relative Energy Error: {abs(final_energy - init_energy)/abs(init_energy):.2e}")

Final Total Energy: -0.73203
Relative Energy Error: 2.29e-04


In [13]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set up the animation figure
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Pre-calculate plot limits to keep the camera still
all_pos = history_positions
ax.set_xlim(all_pos[:, :, 0].min(), all_pos[:, :, 0].max())
ax.set_ylim(all_pos[:, :, 1].min(), all_pos[:, :, 1].max())
ax.set_zlim(all_pos[:, :, 2].min(), all_pos[:, :, 2].max())

ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Position')
ax.set_title('N-Body Simulation 3D Trajectory Animation')

num_particles = all_pos.shape[1]
lines = []
points = []
for i in range(num_particles):
    line, = ax.plot([], [], [], label=f'Particle {i+1}')
    point, = ax.plot([], [], [], 'o')
    lines.append(line)
    points.append(point)

def init():
    for line, point in zip(lines, points):
        line.set_data([], [])
        line.set_3d_properties([])
        point.set_data([], [])
        point.set_3d_properties([])
    return lines + points

def update(frame):
    # Draw traces up to the current frame
    for i, (line, point) in enumerate(zip(lines, points)):
        # The trace line
        line.set_data(all_pos[:frame, i, 0], all_pos[:frame, i, 1])
        line.set_3d_properties(all_pos[:frame, i, 2])
        # The current head particle position
        point.set_data([all_pos[frame-1, i, 0]], [all_pos[frame-1, i, 1]])
        point.set_3d_properties([all_pos[frame-1, i, 2]])
    return lines + points

ax.legend()

num_frames = all_pos.shape[0]
ani = FuncAnimation(fig, update, frames=num_frames, init_func=init, blit=True, interval=20)
plt.close(fig) # Prevent static plot from showing

# Display as HTML5 video
HTML(ani.to_html5_video())